## There are numerous decisions for me to make (very quickly):

* How to present results of classifier so that they do not bias viewer's interpretation of greyscale images. (If I use color, viewer is likely as not to be tricked by the colors into believing there are differences where labeled, even if not merited by the underlying data.
* Should I attempt iForest anomaly detection on images? **Yes!**
  * Advantage: (there exists code) [there exists code](http://stackoverflow.com/questions/30080491/isolation-forest-algorithm-in-python "code provided by user on stackoverflow")
  * Disadvantages: author claims that it is slow, it is not well tested. *Seems to work.*
  * That same reference points to sklearn github pull-request for [a model of increasing maturity](https://github.com/scikit-learn/scikit-learn/pull/4163 "not yet ready for sklearn"), but not yet fully working and not super-well-tested. Another month or two?
  * To impliment iForest, would need to extract feature weights from trained model. Can I do this with Dato's imagenet classifier? **Yes! However, there are devils in the details to be worked out.**
  
### Regarding presentation:
  
  * For minimum viable product:
    * Use [Flask-Uploads](https://pythonhosted.org/Flask-Uploads/) to allow a user to upload a directory filled with images.
    * Apply iForest and rank by anomaly score
    * Present a table with columns [256 x 256 images, file names, anomaly score], ranked by anomaly score.

  * If can't work out NN features, do minimum minimum viable product using iForest on pixel values.
  
### Things to implement:

  * Make sure that can work with color photos as well as grayscale. Should app to a detect on each file and convert to that which is most consistent with others in directory?
  * Would be great to add audio, converting to FFT, clipping or repeating to a nominal length.
  * 

In [1]:
import random
import time
import numpy as np
from numpy.random import normal
import matplotlib.pyplot as plt

from code import iForest as isof

% matplotlib inline

### try iForest using real image data (raw pixels):

In [28]:
import cv2
import os
import glob
#WORK_DIR = '/home/wilber/work/Galvanize/gcp-data/iForest/'
WORK_DIR = '/home/wilber/work/Galvanize/gcp-data/iForest/loonie'
pngCt = len(glob.glob1(WORK_DIR,"*.png"))
print "pngCt: {0}".format(pngCt)
%cd /home/wilber/work/Galvanize/gcp-data/iForest/loonie
Xnames = [""]*pngCt

grayscale = True
if grayscale:
    Xraw = np.full((pngCt, 10000), np.nan)
else:
    Xraw = np.full((pngCt, 30000), np.nan)

for i, file in enumerate(os.listdir(WORK_DIR)):
    if grayscale:
        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    else:
        img = cv2.imread(file)
    if i % 100 == 0:
        print "{0}\t{1}\t{2}".format(i, file, type(img))
    Xraw[i,:] = img.flatten()
    Xnames[i] = file
print Xraw.shape
print Xraw[:10, :20]
Xnames[:10]

pngCt: 2064
/home/wilber/work/Galvanize/gcp-data/iForest/loonie
0	hdll020+0300+0126.png	<type 'numpy.ndarray'>
100	wshdll020+0740+0086.png	<type 'numpy.ndarray'>
200	ldlm020+0620+0086.png	<type 'numpy.ndarray'>
300	hdmm000+0460+0166.png	<type 'numpy.ndarray'>
400	nthdmm020+0340+0166.png	<type 'numpy.ndarray'>
500	wshdll000+1500+0126.png	<type 'numpy.ndarray'>
600	hdll020+0220+0086.png	<type 'numpy.ndarray'>
700	wshdmm020+1180+0166.png	<type 'numpy.ndarray'>
800	ntldlm020+0860+0166.png	<type 'numpy.ndarray'>
900	ldlm020+0780+0166.png	<type 'numpy.ndarray'>
1000	ldlm020+1620+1086.png	<type 'numpy.ndarray'>
1100	ldlm000+1060+0166.png	<type 'numpy.ndarray'>
1200	hdlr020+0300+0166.png	<type 'numpy.ndarray'>
1300	nthdlr020+0500+0166.png	<type 'numpy.ndarray'>
1400	ntldlm020+1140+1046.png	<type 'numpy.ndarray'>
1500	wshdlr000+0540+0166.png	<type 'numpy.ndarray'>
1600	hdll020+0460+0126.png	<type 'numpy.ndarray'>
1700	wsldlm000+1460+0126.png	<type 'numpy.ndarray'>
1800	hdlr020+1300+1006.png	<ty

['hdll020+0300+0126.png',
 'wsldlm000+0820+0166.png',
 'nthdmm000+1020+1006.png',
 'ntldlm000+1540+0166.png',
 'ntldlm000+1020+0166.png',
 'nthdlr020+0260+0046.png',
 'ntldlm020+1020+1086.png',
 'nthdll020+1340+1086.png',
 'wshdll020+0500+0086.png',
 'wshdlr000+1380+0126.png']

In [3]:
%pwd

u'/home/wilber/work/Galvanize/gcp-data/iForest/loonie'

### High-level features from Dato's ImageNet classifier:

In [4]:
import graphlab as gl
WORKING_DIR = '/home/wilber/work/Galvanize/gcp-data/iForest/loonie'
data = gl.image_analysis.load_images(WORKING_DIR, \
                                     random_order=True)
data['image'] = gl.image_analysis.resize(data['image'], 256, 256)

[INFO] This non-commercial license of GraphLab Create is assigned to wilber@ssl.berkeley.eduand will expire on September 15, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-15433 - Server binary: /home/wilber/work/Galvanize/gcp/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1442878614.log
[INFO] GraphLab Server Version: 1.5.2


Couldn't import dot_parser, loading of dot files will not be possible.


In [20]:
names = data['path']
names = map(lambda x: x.split('/')[-1], names)
names[:10]

['nthdlr020+1260+1126.png',
 'hdlr020+1620+1006.png',
 'nthdll000+1580+1006.png',
 'nthdll020+1340+1046.png',
 'nthdll020+0820+0166.png',
 'wshdmm000+0260+0166.png',
 'nthdmm020+1300+1086.png',
 'nthdll020+1420+1126.png',
 'wshdmm000+1380+0126.png',
 'hdll000+0580+0166.png']

In [ ]:
# Import data from MNIST
#data = gl.SFrame('http://s3.amazonaws.com/dato-datasets/mnist/sframe/train6k')

# Create a DeepFeatureExtractorObject

start = time.time()
extractor = gl.feature_engineering.DeepFeatureExtractor(feature = 'image')
print help(extractor)
# Fit the encoder for a given dataset.
time1 = time.time()
print "\n\n", time1 - start
extractor = extractor.fit(data)
time2 = time.time()
print "\n\n", time2 - time1

# Return the model used for the deep feature extraction.
extracted_model = extractor['model']
time3 = time.time()
print "\n\n", time3 - time2

# Extract features.
features_sf = extractor.transform(data)
features_sf.head()
time4 = time.time()
print "\n\n", time4 - time3, ", total time: ", time4 - start

[INFO] This non-commercial license of GraphLab Create is assigned to wilber@ssl.berkeley.eduand will expire on September 15, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-12514 - Server binary: /usr/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1442875412.log
[INFO] GraphLab Server Version: 1.5.2


In [ ]:
#feature = features_sf.head['deep_features_image']
features_sf.num_rows(), features_sf.num_cols()

In [42]:
extractor['model']

Class               : NeuralNetClassifier

Schema
------
Examples            : 1281166
Features            : 1
Target column       : label

Training Summary
----------------
Training accuracy   : 0.625
Validation accuracy : None
Training recall@5   : 0.8512
Validation recall@5 : None
Training time (sec) : 288884.0298

In [43]:
opts_dict = extractor.get_current_options()
for key, value in opts_dict.iteritems():
    print key, value

model Class               : NeuralNetClassifier

Schema
------
Examples            : 1281166
Features            : 1
Target column       : label

Training Summary
----------------
Training accuracy   : 0.625
Validation accuracy : None
Training recall@5   : 0.8512
Validation recall@5 : None
Training time (sec) : 288884.0298



In [39]:
repr(extractor)

'Class               : DeepFeatureExtractor\n\nModel fields\n------------\nFeature(s)          : image\nOutput Column Name  : deep_features_image\n'

In [13]:
features_sf.column_names()

['path', 'image', 'deep_features_image']

In [15]:
np.shape(features_sf['deep_features_image'])

(2064, 4096)

In [19]:
X = np.array(features_sf['deep_features_image'])

In [32]:
X[:5]

array([[ 0.        ,  3.72930121,  0.        , ...,  0.        ,
         0.94428813,  0.        ],
       [ 0.        ,  1.59159303,  0.        , ...,  0.        ,
         0.6683253 ,  0.60917962],
       [ 1.48337448,  0.        ,  0.65555978, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  2.86736774,  0.        , ...,  0.        ,
         1.75487328,  0.        ],
       [ 1.57783175,  1.64519286,  0.        , ...,  0.        ,
         1.5495981 ,  0.        ]])

In [62]:
model = isof.iForest(n_estimators=200)
start = time.time()
model.fit(X)
end = time.time()
print end - start

369.748157024


In [51]:
anom_scores = model.anomaly_score_
sort_indices = np.argsort(anom_scores)

for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	wsldlm000+1340+0126.png	0.615487381279
2	nthdll000+0820+0166.png	0.590250183857
3	hdlr020+0980+0166.png	0.566325593441
4	nthdlr020+1100+1046.png	0.549250004206
5	hdmm020+1020+0166.png	0.488050215793
6	ldlm020+0300+0166.png	0.474663087353
7	wshdmm020+0900+0086.png	0.459938195728
8	nthdll020+1180+1126.png	0.455982553333
9	wshdll020+0420+0086.png	0.452169702796
10	hdmm020+0340+0126.png	0.451372111184
11	nthdmm020+0820+1006.png	0.448498633964
12	wshdmm020+0620+0126.png	0.446066758839
13	hdlr020+0540+0046.png	0.441880783274
14	ntldlm000+0300+1006.png	0.440740666117
15	hdll000+0460+0166.png	0.440144821711


478	fig01.png	0.322530355659
737	fig03.png	0.301604729085
1165	fig00.png	0.275853525065
1850	fig02.png	0.234054963318


### Different layers from pre-trained model:

#### Layer 22

In [5]:
import graphlab as gl
start = time.time()
pretrained_model = gl.load_model('http://s3.amazonaws.com/dato-datasets/deeplearning/imagenet_model_iter45')
time1 = time.time()
pretrained_model = gl.load_model('/home/wilber/work/Galvanize/gcp/DatoImageNetIter45.bin')
print "dt1: ", time1 - start
data['extracted_features'] = pretrained_model.extract_features(data, 22)
print time.time() - time1
totsecs = time.time() - start
hours = int(totsecs/3600)
mins = int((totsecs - 3600.*hours)/60)
secs = totsecs - 3600.*hours - 60.*mins
print "Elapsed time = {0} hours, {1} minutes, {2} seconds".format(hours, mins, secs)

PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/deeplearning/imagenet_model_iter45/dir_archive.ini to /var/tmp/graphlab-wilber/15433/000000.ini
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/deeplearning/imagenet_model_iter45/objects.bin to /var/tmp/graphlab-wilber/15433/000001.bin
705.815214157
807.152285814
Elapsed time = 0 hours, 25 minutes, 12.9680931568 seconds


In [ ]:
#pretrained_model.save('/home/wilber/work/Galvanize/gcp/DatoImageNetIter45.bin')

In [7]:
X22 = np.array(data['extracted_features'])
print np.shape(X22)
X22[:5]

(2064, 1000)


array([[ 0.50253224, -1.25021994, -1.01370823, ..., -1.11672926,
         2.31823492, -1.5534209 ],
       [ 1.77320004, -1.1280663 , -0.63177204, ...,  0.47384962,
         2.69098949, -3.60191417],
       [ 2.16725302, -0.25370082,  2.50009012, ...,  0.80002779,
         1.53945363, -3.46016431],
       [ 1.19407737,  0.69893199, -0.22904766, ..., -1.28837085,
         1.00509715, -0.897053  ],
       [ 1.75302529,  1.67573178, -0.00981579, ..., -0.81604373,
         1.33047044, -1.6480968 ]])

In [8]:
m22 = isof.iForest(n_estimators=200)
start = time.time()
m22.fit(X22)
end = time.time()
print "elapsed time: {0}s.".format(end - start)

Entering iForest.fit() ...

9.99571299553


In [16]:
anom_scores = m22.anomaly_score_
sort_indices = np.argsort(anom_scores)

for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	fig03.png	0.734537036602
2	fig00.png	0.693802240618
3	fig01.png	0.67009517741
4	fig02.png	0.636144819234
5	hdmm000+1700+0166.png	0.596015265989
6	hdlr020+1380+1006.png	0.580211618972
7	nthdlr020+0460+0086.png	0.566919199764
8	hdlr020+1340+1046.png	0.563505802689
9	hdll020+1500+1086.png	0.559962149965
10	ntldlm000+0180+1006.png	0.550150395502
11	hdlr020+1340+1006.png	0.549774212744
12	wsldlm000+0900+0166.png	0.548775950881
13	nthdlr000+1500+1006.png	0.546197986807
14	wsldlm000+1220+0166.png	0.540605083294
15	wsldlm000+1220+0126.png	0.540116587238


1	fig03.png	0.734537036602
2	fig00.png	0.693802240618
3	fig01.png	0.67009517741
4	fig02.png	0.636144819234


### layer 21

In [10]:
start = time.time()
#pretrained_model = gl.load_model('http://s3.amazonaws.com/dato-datasets/deeplearning/imagenet_model_iter45')
#time1 = time.time()
#print time1 - start
data['extracted_features'] = pretrained_model.extract_features(data, 21)
totsecs = time.time() - start
print totsecs
hours = int(totsecs/3600)
mins = int((totsecs - 3600.*hours)/60)
secs = totsecs - 3600.*hours - 60.*mins
print "Elapsed time = {0} hours, {1} minutes, {2} seconds".format(hours, mins, secs)

1690.17795682
Elapsed time = 0 hours, 13 minutes, 8.29797792435 seconds


In [29]:
X21 = np.array(data['extracted_features'])
print np.shape(X21)
X21[:5]

(2064, 4096)


array([[ 0.        ,  3.72930121,  0.        , ...,  0.        ,
         0.94428813,  0.        ],
       [ 0.        ,  1.59159303,  0.        , ...,  0.        ,
         0.6683253 ,  0.60917962],
       [ 1.48337448,  0.        ,  0.65555978, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  2.86736774,  0.        , ...,  0.        ,
         1.75487328,  0.        ],
       [ 1.57783175,  1.64519286,  0.        , ...,  0.        ,
         1.5495981 ,  0.        ]])

In [21]:
m21 = isof.iForest(n_estimators=200)
start = time.time()
m21.fit(X21)
end = time.time()
print "elapsed time: {0}s.".format(end - start)

Entering iForest.fit() ...

elapsed time: 52.0787329674s.


In [30]:
anom_scores = m21.anomaly_score_
sort_indices = np.argsort(anom_scores)

for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	fig01.png	0.643959719912
2	fig03.png	0.631997253255
3	fig00.png	0.595435446863
4	fig02.png	0.532603714266
5	hdmm000+1700+0166.png	0.488955551861
6	hdll020+1460+1086.png	0.486480819583
7	hdmm020+1540+1086.png	0.48042025209
8	wshdlr020+0180+0126.png	0.462428910643
9	hdll020+1420+1086.png	0.457481063087
10	wshdmm020+1740+1006.png	0.456926670075
11	nthdlr020+0460+0086.png	0.454278094795
12	ntldlm000+0180+1006.png	0.448086825148
13	ntldlm000+0220+1006.png	0.445557574545
14	hdlr020+1340+1006.png	0.439459032252
15	nthdlr000+0340+1006.png	0.437586534955


1	fig01.png	0.643959719912
2	fig03.png	0.631997253255
3	fig00.png	0.595435446863
4	fig02.png	0.532603714266


### Layer 20

In [31]:
start = time.time()
# pretrained_model = gl.load_model('http://s3.amazonaws.com/dato-datasets/deeplearning/imagenet_model_iter45')
# time1 = time.time()
# print time1 - start
data['extracted_features'] = pretrained_model.extract_features(data, 20)
print time.time() - time1
totsecs = time.time() - start
hours = int(totsecs/3600)
mins = int((totsecs - 3600.*hours)/60)
secs = totsecs - 3600.*hours - 60.*mins
print "Elapsed time = {0} hours, {1} minutes, {2} seconds".format(hours, mins, secs)

4243.41244578
Elapsed time = 0 hours, 13 minutes, 14.2290129662 seconds


In [32]:
X20 = np.array(data['extracted_features'])
print np.shape(X20)
X20[:5]

(2064, 4096)


array([[ 0.        ,  3.72930121,  0.        , ...,  0.        ,
         0.94428813,  0.        ],
       [ 0.        ,  1.59159303,  0.        , ...,  0.        ,
         0.6683253 ,  0.60917962],
       [ 1.48337448,  0.        ,  0.65555978, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  2.86736774,  0.        , ...,  0.        ,
         1.75487328,  0.        ],
       [ 1.57783175,  1.64519286,  0.        , ...,  0.        ,
         1.5495981 ,  0.        ]])

In [33]:
m20 = isof.iForest(n_estimators=200)
start = time.time()
m20.fit(X20)
end = time.time()
print "elapsed time: {0}s.".format(end - start)

Entering iForest.fit() ...

elapsed time: 48.0057470798s.


In [34]:
anom_scores = m20.anomaly_score_
sort_indices = np.argsort(anom_scores)

for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	fig03.png	0.590907078032
2	fig01.png	0.583476048129
3	fig00.png	0.575494741165
4	fig02.png	0.542881674059
5	hdmm000+1700+0166.png	0.495458899922
6	hdll020+1460+1086.png	0.477883100839
7	wshdlr020+0180+0126.png	0.477283511358
8	hdll020+1420+1086.png	0.471341710934
9	ntldlm000+1020+1006.png	0.46691845264
10	hdlr020+1340+1006.png	0.46166223562
11	nthdll020+1020+1046.png	0.459579220657
12	hdll000+1260+0166.png	0.458959097771
13	ntldlm000+0180+1006.png	0.455783268035
14	nthdmm020+1380+1126.png	0.454901907388
15	ldlm000+1060+0166.png	0.453445736703


1	fig03.png	0.590907078032
2	fig01.png	0.583476048129
3	fig00.png	0.575494741165
4	fig02.png	0.542881674059


### Layer 19

In [35]:
start = time.time()
# pretrained_model = gl.load_model('http://s3.amazonaws.com/dato-datasets/deeplearning/imagenet_model_iter45')
# time1 = time.time()
# print time1 - start
data['extracted_features'] = pretrained_model.extract_features(data, 19)
print time.time() - time1
totsecs = time.time() - start
hours = int(totsecs/3600)
mins = int((totsecs - 3600.*hours)/60)
secs = totsecs - 3600.*hours - 60.*mins
print "Elapsed time = {0} hours, {1} minutes, {2} seconds".format(hours, mins, secs)

5336.73123693
Elapsed time = 0 hours, 13 minutes, 12.6306271553 seconds


In [37]:
X19 = np.array(data['extracted_features'])
print np.shape(X19)
X19[:5]

(2064, 4096)


array([[-1.09079742,  3.72930121, -2.77852869, ..., -4.05487823,
         0.94428813, -0.31786889],
       [-0.76935959,  1.59159303, -3.82672143, ..., -3.59154534,
         0.6683253 ,  0.60917962],
       [ 1.48337448, -0.64070177,  0.65555978, ..., -0.35795367,
        -0.59973013, -1.34989905],
       [-0.27250659,  2.86736774, -0.91383731, ..., -4.791646  ,
         1.75487328, -2.50466108],
       [ 1.57783175,  1.64519286, -0.28011394, ..., -3.5872457 ,
         1.5495981 , -2.22699523]])

In [38]:
m19 = isof.iForest(n_estimators=200)
start = time.time()
m19.fit(X19)
end = time.time()
print "elapsed time: {0}s.".format(end - start)

Entering iForest.fit() ...

elapsed time: 27.300606966s.


In [39]:
anom_scores = m19.anomaly_score_
sort_indices = np.argsort(anom_scores)
for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	nthdll000+0900+1006.png	0.722790433726
2	nthdlr000+1500+1006.png	0.673196322709
3	wshdll000+0860+0166.png	0.66657059519
4	nthdlr020+1460+1006.png	0.651122223423
5	nthdll020+1460+1006.png	0.648013855295
6	nthdlr000+1540+1006.png	0.640718432395
7	wshdll020+0420+0046.png	0.634636544086
8	nthdll000+0940+1006.png	0.632354279033
9	fig02.png	0.630918772611
10	nthdll020+1420+1006.png	0.628112758187
11	fig00.png	0.628013312611
12	hdll020+0460+0166.png	0.607640818237
13	nthdlr020+1340+1206.png	0.604512939923
14	wshdll020+0500+0086.png	0.601905178846
15	nthdll020+0780+1006.png	0.601754753757


9	fig02.png	0.630918772611
11	fig00.png	0.628013312611
17	fig03.png	0.597391174453
32	fig01.png	0.571525568133


### Have gone too far!

### Now try with raw pixels:

In [3]:
model = isof.iForest(n_estimators=200)
start = time.time()
model.fit(Xraw)
end = time.time()
print end - start

Entering iForest.fit() ...

63.1782519817


In [4]:
anom_scores = model.anomaly_score_
sort_indices = np.argsort(anom_scores)

for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	fig03.png	0.616293966178
2	fig01.png	0.57686392225
3	nthdll000+0900+1006.png	0.573929882553
4	hdll020+1420+1086.png	0.553295278689
5	nthdll020+0820+1006.png	0.553048603554
6	fig00.png	0.551870372783
7	hdll020+1460+1086.png	0.551777778897
8	hdll020+1220+1006.png	0.548237328637
9	nthdll020+1260+1126.png	0.547619981709
10	hdll020+1260+1006.png	0.546107621288
11	nthdll000+1020+1006.png	0.54555213689
12	nthdll020+1260+1166.png	0.545324321227
13	nthdll000+0940+1006.png	0.530078808273
14	nthdll020+1140+1086.png	0.529312284063
15	nthdll020+1420+1046.png	0.52612597045


1	fig03.png	0.616293966178
2	fig01.png	0.57686392225
6	fig00.png	0.551870372783
72	fig02.png	0.484954116883
